In [6]:
# problem
#f(x) = 3x1 + 2x2 + 4x3 + 2x4 + 5x5 = 100

In [1]:
import numpy as np
import random
from collections import Counter

In [18]:
best_solution, best_fitness = genetic_algorithm(
    population_size=5,
    num_genes=5,
    num_parents=3,
    mutation_rate=0.1,
    num_generetions=10,
    cross_over_rate = 0.45
)

Population:
 [[2, 1, 8, 6, 1], [4, 2, 4, 3, 7], [8, 3, 1, 3, 1], [3, 1, 0, 5, 1], [3, 8, 3, 6, 4]]
Best population after 10: 
 [[4, 2, 1, 3, 1], [2, 1, 8, 6, 7], [3, 6, 4, 6, 4], [6, 8, 3, 3, 3], [8, 9, 1, 3, 7]]

Best solution after 10 generations:
Best Chromosome: [2, 1, 8, 6, 7] with Fitness: 87


In [13]:
def genetic_algorithm(population_size, num_genes, num_parents, mutation_rate, num_generetions, cross_over_rate):
    population = generate_population(num_genes, population_size)
    y = 0
    for generation in range(num_generetions):
        fitness_scores = objective_function(population)
        best_chromosome = population[fitness_scores.index(max(fitness_scores))]
        fitness_scores = fitness_function(fitness_scores)
        probability_score = probability_of_fitness(fitness_scores)
        cumulative_score = cumulative_probability(probability_score)
        parents = selection(cumulative_score, num_parents, population, cross_over_rate, population_size)
        # Step 4: Apply crossover to generate new offspring
        offspring = []
        unique_parents = list(set(tuple(parent) for parent in parents))
        unique_parents = [list(parent) for parent in unique_parents]
       
        for i in range(0, len(unique_parents), 2):
            parent1 = unique_parents[i]
            parent2 = unique_parents[(i + 1) % len(unique_parents)]  # Loop back if odd number of parents
            mutation1, mutation2 = mutations_generate(parent1, parent2)
            offspring.append(mutation1)
            offspring.append(mutation2)

        for x in unique_parents:
            population.remove(x)
        
 # Step 5: Apply mutation to the offspring
        mutated_offspring = [mutation(child, mutation_rate) for child in offspring]
        # Step 6: Replace the population with the new generation (mutated offspring)
        for i in mutated_offspring:
            if(len(population) < population_size):
                population.append(i)

        y += 1
        final_fitness_scores = objective_function(population)
        if 95 < max(final_fitness_scores) <= 100:
            break


    # Final result: Return the best solution found after all generations
    
    best_fitness = max(final_fitness_scores)
    best_chromosome = population[final_fitness_scores.index(best_fitness)]
    print(f"Best population after {y}: \n {population}")
    print(f"\nBest solution after {y} generations:")
    print(f"Best Chromosome: {best_chromosome} with Fitness: {best_fitness}")

    return best_chromosome, best_fitness
        


In [3]:
def generate_population(num_of_genes, num_of_individuals):
    population = []
    for i in range(num_of_individuals):
        chrmosome = random.choices(range(0, 9), k=5)
        population.append(chrmosome)
    print('Population:\n',population)
    return population

In [4]:
def objective_function(population):
    fitness_score = []
    values = [3, 2, 4, 2, 5]
    x = 0
    for i in range(len(population)):
        total = 0
        chromosome = population[i]
        for j in chromosome:
            total = total + values[x]*j
            x = x + 1
        x = 0
        fitness_score.append(total)
    return fitness_score

In [5]:
def fitness_function(objective_total):
    fitness_score = []
    for i in objective_total:
        fitness_score.append(1/(i + 1))
    return fitness_score

In [6]:
def probability_of_fitness(fitness_score):
    total_fitness = sum(fitness_score)
    # print("Total fitness:", total_fitness)
    probability_score = []
    for i in fitness_score:
        probability_score.append(i/total_fitness)
    return probability_score

In [7]:
def cumulative_probability(probability_score):
    cumulative_score = []
    cumulative_total = 0
    for i in probability_score:
        cumulative_total = cumulative_total + i
        cumulative_score.append(cumulative_total)
    return cumulative_score

In [8]:
def random_number_generate(cross_over_rate):
    random_numbers = []
    num = [random.uniform(0, 1) for _ in range(5)]
    for _ in range(5):
        num = random.uniform(0, 1)
        if(num <= cross_over_rate):
            random_numbers.append(num)
    return random_numbers

In [9]:
from collections import Counter

def selection(cumulative_score, num_parents, population, cross_over_rate, size):
    selected = []
    random_numbers = random_number_generate(cross_over_rate)
    
    # Select parents based on cumulative scores and random numbers
    for _ in range(num_parents):
        first = 0.0
        for i in range(len(cumulative_score)):
            second = cumulative_score[i]
            for j in range(len(random_numbers)):
                if first < random_numbers[j] < second:
                    selected.append(population[cumulative_score.index(second)])
            first = cumulative_score[i]
    
    # Count occurrences of each sublist in selected
    counts = Counter(tuple(sublist) for sublist in selected)
    
    # Sort sublists by occurrence count in descending order
    sorted_selected = sorted(counts.items(), key=lambda x: x[1], reverse=True)

    updated_population = list(population)
    population = []

    # Fill population with the top sublists based on occurrence count
    for sublist, count in sorted_selected[:size]:
        # print("Sublist:", sublist, " count:", count)
        sublist_as_tuple = tuple(sublist)  # Use tuple for consistent comparison
        if sublist_as_tuple in updated_population:  # Check if the sublist exists
            updated_population.remove(sublist_as_tuple)  # Remove if exists
        
        for _ in range(count):
            if len(population) < size:
                population.append(list(sublist))  # Append the list version of the sublist


    # Fill remaining slots from updated_population if needed
    while len(population) < size and updated_population:
        x = updated_population.pop(0)  # Pop the first element safely
        population.append(x)  # Append it to population

    # print("Final Population: ", population)
    # print("Remaining from updated_population: ", len(updated_population))

    return population[:size]  # Ensure we only return up to 5 elements


In [10]:
def mutations_generate(parent1, parent2):
    # Ensure that the crossover index stays within the bounds of the chromosome length
    crossover_index = random.randint(1, len(parent1) - 1)  # Make sure it's not 0 or the full length
    # print(f"Random Crossover Index: {crossover_index}")
  
    # Perform the crossover, ensuring both offspring have the same length as parents
    parent1_front = parent1[:crossover_index]
    parent1_back = parent1[crossover_index:]
    parent2_front = parent2[:crossover_index]
    parent2_back = parent2[crossover_index:]
    
    # Combine the parts to form two new offspring
    mutation1 = parent1_front + parent2_back
    mutation2 = parent2_front + parent1_back

    # Make sure both offspring have the same length as parents
    if len(mutation1) != len(parent1):
        print("Error: Length mismatch in mutation1")
    if len(mutation2) != len(parent2):
        print("Error: Length mismatch in mutation2")

    # print(mutation1, mutation2)

    return mutation1, mutation2


    

In [11]:
def mutation(chromosome, mutation_rate=0.01):
    # Loop through each gene in the chromosome
    for i in range(len(chromosome)):
        # With a probability equal to mutation_rate, flip the gene
        if random.random() < mutation_rate:
            chromosome[i] = random.randint(0, 9)  # Randomly change the gene
    return chromosome
